In [ ]:
import  numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeCV

from sklearn.preprocessing import QuantileTransformer, quantile_transform

from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error
# ....

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
# import lightgbm as lgb 

from lightgbm import LGBMRegressor,LGBMClassifier
from sklearn.pipeline import Pipeline, make_pipeline

from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime

# tuner.get_best_hyperparameters(num_trials=1)[0].values
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from datetime import datetime
import pickle5 as pickle



%matplotlib inline

### Loading dummy variable features

In [ ]:
# Load cleaned dummy variable data
with open('cleaned_data_dummy_vars.pickle', 'rb') as handle:
    background_imputed_tot = pickle.load(handle)
    
with open('X_train_dummy_vars.pickle', 'rb') as handle:
    X_train = pickle.load(handle)

with open('X_CV_dummy_vars.pickle', 'rb') as handle:
    X_CV = pickle.load(handle)

with open('x_test_dummy_vars.pickle', 'rb') as handle:
    x_test = pickle.load(handle)
    
with open('x_leaderboard_dummy_vars.pickle', 'rb') as handle:
    x_leaderboard = pickle.load(handle)
    
with open('y_train_dummy_vars.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
    
with open('y_CV_dummy_vars.pickle', 'rb') as handle:
    y_CV = pickle.load(handle)
    
background = pd.read_csv('FFChallenge_v5/background.csv', sep=',', header=0,index_col=0,low_memory=False)
# train.csv contains 2,121 rows (one per child in the training set) and 7 columns.
train = pd.read_csv('FFChallenge_v5/train.csv', sep=',', header=0, index_col=0,low_memory=False)
########### Holdout dataset for internal testing only
test = pd.read_csv('test.csv',header=0, index_col=0,low_memory=False)
leaderboard = pd.read_csv('leaderboard.csv', header=0, index_col=0,low_memory=False)
leaderboard = leaderboard.dropna()

In [ ]:
# Final stats and useful variables associated with each column

numerical_columns = [c for c,v in background_imputed_tot.dtypes.iteritems() if v in [np.float,np.int,np.int64]]
non_numerical_columns = [c for c,v in background_imputed_tot.dtypes.iteritems() if v not in [np.float,np.int,np.int64]]
continuous_cols_lst = list()
continuous_cols_lst = background_imputed_tot.T.loc[(background_imputed_tot.apply(pd.Series.nunique) >= 15).values==True].index.to_list()

print('Number of continuous columns %s' % len(continuous_cols_lst))
# background = background[numerical_columns]
# background.head()

Number of continuous columns 507


## Creating regression neural network model for continuous outcomes

In [ ]:
# Creating a neural network with 1 hidden layer, where the three hyperparameters to be tunred are "num_hidden_layer_1","dropout_layer_1" and the learning_rate
# Uses Adam optimizer and mean squared error loss
# Uses a linear layer for the final layer for continuous predictions
def make_regression_nn(num_hidden_layer_1,dropout_layer_1,learning_rate):

    model = Sequential()
    # Defining input layer, with size of input feature (size of image is 784)
    model.add(Dense(num_hidden_layer_1, input_shape=(len(X_train_normalized.columns),)))
    model.add(Activation('sigmoid'))     
    # Using a dropout rate of 20% for regularization for each layer
    model.add(Dropout(dropout_layer_1))

    

    # Defining third hidden layer
    model.add(Dense(1))

    model.compile(optimizer=Adam(
            learning_rate = learning_rate),
            loss='mean_squared_error', metrics=[MeanSquaredError()])#     model_test.summary()
    
    return model

In [ ]:
# variable = 'jobTraining'

from sklearn.model_selection import train_test_split, GridSearchCV
from keras.layers.core import Lambda
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier,KerasRegressor
from tensorflow.keras.metrics import MeanSquaredError
from keras.regularizers import l1
import tensorflow as tf
from sklearn.model_selection import PredefinedSplit
# from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedKFold,RepeatedStratifiedKFold

from numpy.random import seed
seed(0)
import tensorflow
tensorflow.random.set_seed(0)


acc_nns = dict()
best_params_dict = dict()
train_cv_errors = dict()
history_accs = dict()

for variable in ['gpa', 'grit', 'materialHardship']: # Looping through continuous outcomes
    print('---------------variable:',variable,'---------------\n\n\n')
    # Get non-na rows for this specific variable
    y_train_no_na = y_train[variable].dropna()
    X_train_no_na = X_train.loc[y_train_no_na.index.values]

    y_CV_no_na = y_CV.copy()[variable].dropna()
    X_CV_no_na = X_CV.copy().loc[y_CV_no_na.index.values]

    mask = leaderboard[variable].isna()
    x_leaderboard = background_imputed_tot.loc[leaderboard.index.values]
    y_leaderboard = leaderboard[variable]
    x_leaderboard_no_na = x_leaderboard.loc[y_leaderboard.index]

    y_train_trans= y_train_no_na.copy()
    y_CV_trans = y_CV_no_na.copy()
    y_test_trans = test[variable].dropna()


    x_test_no_na = x_test.loc[y_test_trans.index.values]



    ##### Normalize features to be between 0 and 1 and apply the same transformation on the validation and test sets
    # Pandas applies these operations row-wise
    X_train_normalized = X_train_no_na.copy()
    X_cv_normalized = X_CV_no_na.copy()
    x_leaderboard_normalized = x_leaderboard_no_na.copy()




    ##### Normalize features to be between 0 and 1 and apply the same transformation on the validation and test sets
    # Pandas applies these operations row-wise
    X_train_normalized = X_train_no_na.copy()
    X_cv_normalized = X_CV_no_na.copy()
    x_leaderboard_normalized = x_leaderboard_no_na.copy()
    x_test_normalized = x_test_no_na.copy()


    # Getting normalization parameters
    column_min_normalization = X_train_no_na[continuous_cols_lst].min()
    column_max_normalization = X_train_no_na[continuous_cols_lst].max()

    # Normalizing features
    X_train_normalized[continuous_cols_lst]=(X_train_no_na[continuous_cols_lst]-column_min_normalization)\
    /(column_max_normalization-column_min_normalization)

    X_cv_normalized[continuous_cols_lst]=(X_CV_no_na[continuous_cols_lst]-column_min_normalization)\
    /(column_max_normalization-column_min_normalization)

    x_leaderboard_normalized[continuous_cols_lst] = (x_leaderboard_no_na[continuous_cols_lst]-column_min_normalization)\
    /(column_max_normalization-column_min_normalization)

    x_test_normalized[continuous_cols_lst] = (x_test_no_na[continuous_cols_lst]-column_min_normalization)\
    /(column_max_normalization-column_min_normalization)

    classifier = KerasRegressor(make_regression_nn, epochs=500,verbose = 0)

    # Parameter search space (need high values for dropout to reduce overfitting)
    params = [{'num_hidden_layer_1': [64,128,256,512,1024],
            'dropout_layer_1': [0.75,0.8,0.85,0.9,0.95],
            'learning_rate': [0.002,0.0005,0.0001]}]

    # Use grid search with repeated k fold cross validation
    grid = GridSearchCV(classifier,
                          param_grid=params,
                          scoring='neg_mean_squared_error', #sklearn optimizing by maximizing negative brier score
                          #  n_jobs=-1,
                          verbose=0,
                          cv=RepeatedKFold(n_splits=3, n_repeats=2)     # Number of folds for CV
                          #  fit_params={'callbacks': [tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                          #                                          min_delta=0.001, 
                          #                                          patience=25, 
                          #                                          mode='min',
                          #                                          verbose=2)],
                          #             'validation_data': (np.array(X_cv_normalized), np.array(y_CV_trans))
                          #             }
                    )

    grid.fit(np.array(X_train_normalized), np.array(y_train_trans),callbacks= [tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                                  min_delta=0.001, 
                                                                  patience=25, 
                                                                  mode='min',
                                                                  verbose=0)],validation_data= (np.array(X_cv_normalized), np.array(y_CV_trans)))

    errs =grid.cv_results_['mean_test_score']
    print('Gridsearch complete')
    
    # Obtain the best parameters and save them
    best_params = grid.best_params_
    print('best params from gridsearch =',best_params)
    best_params_dict[variable] = best_params
    # model_test = make_model(best_params['num_hidden_layer_1'],best_params['dropout_layer_1'],best_params['learning_rate'])

    num_hidden_layer_1 = best_params['num_hidden_layer_1']
    dropout_layer_1 =best_params['dropout_layer_1']

    learning_rate = best_params['learning_rate'] # jobTraining lr (also works well for layoff)
    # learning_rate = 0.000060947

    
    ###### Train the same neural network architecture from scratch (doesn't seem to work when I call my make regression function, so made it manually here)
    model_test = Sequential()
    # Defining input layer, with size of input feature (size of image is 784)
    model_test.add(Dense(num_hidden_layer_1, input_shape=(len(X_train_normalized.columns),)))
    model_test.add(Activation('sigmoid'))     
    # Using a dropout rate of 20% for regularization for each layer
    model_test.add(Dropout(dropout_layer_1))

    # Defining output layer
    model_test.add(Dense(1))

    model_test.compile(optimizer=Adam(
              learning_rate = learning_rate),
              loss='mean_squared_error', metrics=[MeanSquaredError()])#     model_test.summary()


    # Use early stopping, and save the model performance to be able to load the network parameters 25 iterations before early stopping if it was initiated
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                                  min_delta=0.0001, 
                                                                  patience=25, 
                                                                  verbose=2)
    # Saving parameters at the earliest iteration when validation loss was at a minimum
    mc = ModelCheckpoint('best_model_nn' + variable + '.h5', monitor='val_loss', verbose=1, save_best_only=True)


    history = model_test.fit(X_train_normalized, y_train_trans, epochs=500, validation_split=0.2,\
                    validation_data=(X_cv_normalized, y_CV_trans),\
                      callbacks=[stop_early, mc], verbose = -1
                    )

    history_accs[variable] = history.history

    model_test = load_model('best_model_nn' + variable + '.h5')


    y_pred_train = model_test.predict(X_train_normalized)
    y_pred_all = model_test.predict(x_leaderboard_normalized)
    cv_preds = model_test.predict(X_cv_normalized)
    test_preds = model_test.predict(x_test_normalized)

    # Can use mean square error as performance metric for both, since for 1 output, the mean square error and brier loss are the same.
    print('---- Train error ----')
    print(mean_squared_error(y_train_trans,y_pred_train))
    print('---- CV error ----')
    print(mean_squared_error(y_CV_trans,cv_preds))
    train_cv_errors[variable] = [mean_squared_error(y_train_trans,y_pred_train),mean_squared_error(y_CV_trans,cv_preds)]

    # y_CV_trans = (y_CV_no_na-y_train_mean)/y_train_std
    print('---- leaderboard stats ----')
    print(r2_score(y_leaderboard[~mask],y_pred_all[~mask]))
    print(mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask]))
    print('\n\n\n')
    preds_baseline_test = np.array([y_test_trans.mean()for i in range(len(y_test_trans))])
    print('Baseline test MSE =',mean_squared_error(y_test_trans,preds_baseline_test))

    print('Model test MSE =',mean_squared_error(test_preds,y_test_trans))
    # acc_nns[variable] = mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask])
    print('\n\n\n\n')

---------------variable: gpa ---------------



Gridsearch complete
best params from gridsearch = {'dropout_layer_1': 0.75, 'learning_rate': 0.0005, 'num_hidden_layer_1': 1024}
Epoch 1/500

Epoch 00001: val_loss improved from inf to 0.77314, saving model to best_model_nngpa.h5
Epoch 2/500

Epoch 00002: val_loss improved from 0.77314 to 0.45609, saving model to best_model_nngpa.h5
Epoch 3/500

Epoch 00003: val_loss improved from 0.45609 to 0.42667, saving model to best_model_nngpa.h5
Epoch 4/500

Epoch 00004: val_loss did not improve from 0.42667
Epoch 5/500

Epoch 00005: val_loss improved from 0.42667 to 0.38143, saving model to best_model_nngpa.h5
Epoch 6/500

Epoch 00006: val_loss improved from 0.38143 to 0.37913, saving model to best_model_nngpa.h5
Epoch 7/500

Epoch 00007: val_loss improved from 0.37913 to 0.37835, saving model to best_model_nngpa.h5
Epoch 8/500

Epoch 00008: val_loss did not improve from 0.37835
Epoch 9/500

Epoch 00009: val_loss did not improve from 0.37835
Epoch

## Creating classification neural network model for categorical outcomes

In [ ]:
# Creating a neural network with 1 hidden layer, where the three hyperparameters to be tunred are "num_hidden_layer_1","dropout_layer_1" and the learning_rate
# Uses Adam optimizer and binary cross entropy loss
# Uses a sigmoid activation for the final layer for binary predictions
def make_classification_nn(num_hidden_layer_1,dropout_layer_1,learning_rate):
    model_test = Sequential()
    # Defining input layer, with size of input feature (size of image is 784)
    model_test.add(Dense(num_hidden_layer_1, input_shape=(len(X_train_normalized.columns),)))
    model_test.add(Activation('sigmoid'))     
    # Using a dropout rate of 20% for regularization for each layer
    model_test.add(Dropout(dropout_layer_1))

    

    # Defining third hidden layer
    model_test.add(Dense(1))
    model_test.add(Activation('sigmoid'))

    model_test.compile(optimizer=Adam(
            learning_rate = learning_rate),
            loss='binary_crossentropy', metrics=['accuracy',MeanSquaredError()])#     model_test.summary()
    
    return model_test

In [ ]:
# variable = 'jobTraining'

from sklearn.model_selection import train_test_split, GridSearchCV
from keras.layers.core import Lambda
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier,KerasRegressor
from tensorflow.keras.metrics import MeanSquaredError
from keras.regularizers import l1
import tensorflow as tf
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import RepeatedStratifiedKFold


from numpy.random import seed
seed(0)
import tensorflow
tensorflow.random.set_seed(0)


acc_nns = dict()
best_params_dict = dict()
train_cv_errors = dict()
for variable in ['eviction','jobTraining','layoff']: # Looping through categorical outcomes
    print('---------------variable:',variable,'---------------\n\n\n')
    # Get non-na rows for this specific variable
    y_train_no_na = y_train[variable].dropna()
    X_train_no_na = X_train.loc[y_train_no_na.index.values]

    y_CV_no_na = y_CV.copy()[variable].dropna()
    X_CV_no_na = X_CV.copy().loc[y_CV_no_na.index.values]

    mask = leaderboard[variable].isna()
    x_leaderboard = background_imputed_tot.loc[leaderboard.index.values]
    y_leaderboard = leaderboard[variable]
    x_leaderboard_no_na = x_leaderboard.loc[y_leaderboard.index]

    y_train_trans= y_train_no_na.copy()
    y_CV_trans = y_CV_no_na.copy()
    y_test_trans = test[variable].dropna()


    x_test_no_na = x_test.loc[y_test_trans.index.values]


    ##### Normalize features to be between 0 and 1 and apply the same transformation on the validation and test sets
    # Pandas applies these operations row-wise
    X_train_normalized = X_train_no_na.copy()
    X_cv_normalized = X_CV_no_na.copy()
    x_leaderboard_normalized = x_leaderboard_no_na.copy()




    ##### Normalize features to be between 0 and 1 and apply the same transformation on the validation and test sets
    # Pandas applies these operations row-wise
    X_train_normalized = X_train_no_na.copy()
    X_cv_normalized = X_CV_no_na.copy()
    x_leaderboard_normalized = x_leaderboard_no_na.copy()
    x_test_normalized = x_test_no_na.copy()


    column_min_normalization = X_train_no_na[continuous_cols_lst].min()
    column_max_normalization = X_train_no_na[continuous_cols_lst].max()


    X_train_normalized[continuous_cols_lst]=(X_train_no_na[continuous_cols_lst]-column_min_normalization)\
    /(column_max_normalization-column_min_normalization)

    X_cv_normalized[continuous_cols_lst]=(X_CV_no_na[continuous_cols_lst]-column_min_normalization)\
    /(column_max_normalization-column_min_normalization)

    x_leaderboard_normalized[continuous_cols_lst] = (x_leaderboard_no_na[continuous_cols_lst]-column_min_normalization)\
    /(column_max_normalization-column_min_normalization)

    x_test_normalized[continuous_cols_lst] = (x_test_no_na[continuous_cols_lst]-column_min_normalization)\
    /(column_max_normalization-column_min_normalization)

    classifier = KerasRegressor(make_classification_nn, epochs=500,verbose = 0)

    params = [{'num_hidden_layer_1': [64,128,256],
            'dropout_layer_1': [0.8,0.9,0.95,0.98],
            'learning_rate': [0.0001,0.00005,0.000025]}]


    # Use repeated stratified cross validation to preserve the class distributions
    grid = GridSearchCV(classifier,
                          param_grid=params,
                          scoring='neg_mean_squared_error', #sklearn optimizing by maximizing negative brier score
                          #  n_jobs=-1,
                          verbose=0,
                          cv=RepeatedStratifiedKFold(3,2)# Number of folds for CV
                          #  fit_params={'callbacks': [tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                          #                                          min_delta=0.001, 
                          #                                          patience=25, 
                          #                                          mode='min',
                          #                                          verbose=2)],
                          #             'validation_data': (np.array(X_cv_normalized), np.array(y_CV_trans))
                          #             }
                    )

    grid.fit(np.array(X_train_normalized), np.array(y_train_trans),callbacks= [tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                                  min_delta=0.001, 
                                                                  patience=25, 
                                                                  mode='min',
                                                                  verbose=0)],validation_data= (np.array(X_cv_normalized), np.array(y_CV_trans)))


    print('Gridsearch complete')
    best_params = grid.best_params_

    print('best params from gridsearch =',best_params)
    best_params_dict[variable] = best_params
    # model_test = make_model(best_params['num_hidden_layer_1'],best_params['dropout_layer_1'],best_params['learning_rate'])

    num_hidden_layer_1 = best_params['num_hidden_layer_1']
    dropout_layer_1 =best_params['dropout_layer_1']

    learning_rate = best_params['learning_rate'] # jobTraining lr (also works well for layoff)
    # learning_rate = 0.000060947

     ###### Train the same neural network architecture from scratch (doesn't seem to work when I call my make regression function, so made it manually here)
    model_test = Sequential()
    # Defining input layer, with size of input feature (size of image is 784)
    model_test.add(Dense(num_hidden_layer_1, input_shape=(len(X_train_normalized.columns),)))
    model_test.add(Activation('sigmoid'))     
    # Using a dropout rate of 20% for regularization for each layer
    model_test.add(Dropout(dropout_layer_1))


    # Defining third hidden layer
    model_test.add(Dense(1))
    model_test.add(Activation('sigmoid'))

    model_test.compile(optimizer=Adam(
              learning_rate = learning_rate),
              loss='binary_crossentropy', metrics=['accuracy',MeanSquaredError()])#     model_test.summary()


    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                                  min_delta=0.0001, 
                                                                  patience=25, 
                                                                  verbose=2)
    # Saving parameters at the earliest iteration when validation loss was at a minimum
    mc = ModelCheckpoint('best_model_nnh2' + variable + '.h5', monitor='val_loss', verbose=1, save_best_only=True)


    history = model_test.fit(X_train_normalized, y_train_trans, epochs=500, validation_split=0.2,\
                    validation_data=(X_cv_normalized, y_CV_trans),\
                      callbacks=[stop_early, mc], verbose = -1
                    )

    # model_test = load_model('best_model_nnh2' + variable + '.h5')

    model_test.save('best_model_nn_overall_'+variable+'.h5')

    y_pred_train = model_test.predict(X_train_normalized)
    y_pred_all = model_test.predict(x_leaderboard_normalized)
    cv_preds = model_test.predict(X_cv_normalized)
    test_preds = model_test.predict(x_test_normalized)

    # Can use mean square error as performance metric for both, since for 1 output, the mean square error and brier loss are the same.
    print('---- Train error ----')
    print(mean_squared_error(y_train_trans,y_pred_train))
    print('---- CV error ----')
    print(mean_squared_error(y_CV_trans,cv_preds))
    train_cv_errors[variable] = [mean_squared_error(y_train_trans,y_pred_train),mean_squared_error(y_CV_trans,cv_preds)]

    # y_CV_trans = (y_CV_no_na-y_train_mean)/y_train_std
    print('---- leaderboard stats ----')
    print(r2_score(y_leaderboard[~mask],y_pred_all[~mask]))
    print(mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask]))
    print('\n\n\n')
    preds_baseline_test = np.array([y_test_trans.value_counts()[True]/(y_test_trans.value_counts()[False]+y_test_trans.value_counts()[True]) for i in range(y_test_trans.value_counts()[False]+y_test_trans.value_counts()[True])])

    print('Baseline test brier score =',mean_squared_error(y_test_trans,preds_baseline_test))

    print('Model test brier score =',mean_squared_error(test_preds,y_test_trans))
    acc_nns[variable] = mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask])
    print('\n\n\n\n')

---------------variable: eviction ---------------



Gridsearch complete
best params from gridsearch = {'dropout_layer_1': 0.9, 'learning_rate': 2.5e-05, 'num_hidden_layer_1': 64}
Epoch 1/500

Epoch 00001: val_loss improved from inf to 0.23335, saving model to best_model_nnh2eviction.h5
Epoch 2/500

Epoch 00002: val_loss did not improve from 0.23335
Epoch 3/500

Epoch 00003: val_loss improved from 0.23335 to 0.22964, saving model to best_model_nnh2eviction.h5
Epoch 4/500

Epoch 00004: val_loss improved from 0.22964 to 0.22728, saving model to best_model_nnh2eviction.h5
Epoch 5/500

Epoch 00005: val_loss improved from 0.22728 to 0.22627, saving model to best_model_nnh2eviction.h5
Epoch 6/500

Epoch 00006: val_loss improved from 0.22627 to 0.22407, saving model to best_model_nnh2eviction.h5
Epoch 7/500

Epoch 00007: val_loss did not improve from 0.22407
Epoch 8/500

Epoch 00008: val_loss improved from 0.22407 to 0.22399, saving model to best_model_nnh2eviction.h5
Epoch 9/500

Epoch 00009

## Save the best parameters and the model's training history

In [ ]:
import pickle

with open('best_params_classification_nn.pickle', 'wb') as handle:
    pickle.dump(best_params_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


with open('history_accs_nn.pickle', 'wb') as handle:
    pickle.dump(history_accs, handle, protocol=pickle.HIGHEST_PROTOCOL)
